In [5]:
import torch.nn
import torch.nn.functional as F

In [1]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000
NB_IMAGES = 8000

embed_dim = 256
units = 512
num_steps = NB_IMAGES // BATCH_SIZE

# Extraction from CNN: (64, 2048)
features_shape = 2048
attention_shape = 64

In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self):
        super(EncoderCNN, self).__init__()
        
        self.cnn = None  # init pretrained CNN
        self.fc = nn.Dense(embed_dim)
        

    def forward(self, images):
        # images shape: (8, 8, 2048)
        features = self.cnn.fit(images)  # images shape: (64, 2048)
        features = self.fc.fit(features)  # images shape: (64, 256)
        features = F.relu(features)
        
        return features

In [ ]:
class Attention(nn.Module):
    def __init__(self, units, attention_shape):
        super(Attention, self).__init__()
        
        self.W1 = nn.Linear(units, attention_shape)
        self.W2 = nn.Linear(units, attention_shape)
        
        self.V = nn.Linear(attention_shape, 1)
        
    def forward(self, features, hidden_state):
        # "concat" scores (tanh)
        concat = torch.tanh(
            self.W1(features) + self.W2(hidden_state)
        ) # (units, attention_dim)
        attention_scores = self.V(concat)  # (units, 1)
        
        alignment = F.softmax(attention_scores, dim=1)  # (units)
        
        context_vector = features * alignment.unsqueeze(2)  # (units, features_dim)
        context_vector = context_vector.sum(dim=1)   # (units)
        
        return alignment, context_vector

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self):
        super().__init__()

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = EncoderCNN()
        self.decoder = DecoderRNN()
    
    def forward(self, images, captions):
        features = self.encoder.forward(images)
        outputs = self.decoder.forward(features, captions)
        return outputs